# DEX Functions

In [4]:
import os
import re
import sys
import hashlib
import socket
import requests
import json
import time
from time import time
from operator import itemgetter
from datetime import datetime, timezone
from pyp2p.net import *

from config import Config

# from blockchain import Blockchain

In [2]:
aDict = {'a':1, 'b':2}
str(aDict)

"{'a': 1, 'b': 2}"

### Address Derive System

In [7]:
# 1. Make key pares
# 2. Get address by hashing public key with SHA3 (Keccak) and adding Id bytes
# 3. Sign tx (pub key added) with private key and send it to pool
# 4. Check hash and pub key equality and verify msg signiture
# 5. If step 4 == True submit transaction into pool

#### Create wallet

In [5]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_v1_5

In [20]:
key = RSA.generate(1024)
pubKey = key.publickey().export_key(format='DER')
pubKey.hex()

'30819f300d06092a864886f70d010101050003818d0030818902818100a49e0bf9e0ccb99e28623cfb9b901a7e3e093ed54f477ec4d90d480f0f6df612e40d1a81717c4047f70d23ff3a8ddf7fe67bf907d09871d2c3826541c59b18e4932071b61db6bf492c9c8721efdb92d30d55ad3cf7b1f643723f23c654c58be9cbcfb2bb76c1a1d988e064908f1bc4951d4cec1ebfa9c0a6fcc9d85f3abf893b0203010001'

In [21]:
prKey = key.export_key(format='DER')
prKey.hex()

'3082025d02010002818100a49e0bf9e0ccb99e28623cfb9b901a7e3e093ed54f477ec4d90d480f0f6df612e40d1a81717c4047f70d23ff3a8ddf7fe67bf907d09871d2c3826541c59b18e4932071b61db6bf492c9c8721efdb92d30d55ad3cf7b1f643723f23c654c58be9cbcfb2bb76c1a1d988e064908f1bc4951d4cec1ebfa9c0a6fcc9d85f3abf893b02030100010281800f3a07458fc3a00fbf10872f86ce278531dbcd1212093cf29c591c2738782560cdf3605644d7076320b8f8716d4b1e37486c833673d7060046d152e8688cd61a786a6fef3d006578ef92c90b6080ba3f1bdde58674f647598b2a2d3e41134661af25010d3fb65d943f37c913b4499d00646ffae5b325b52094da4ace000c9c21024100c0480c324e5c47780c6126eff20e72c1f27084eddfd17dd6a1e5eef1434b9d29d53afddff1c8d97e1813350baa884eaf56deea397882855822ff1e2fc0e01753024100db2b273259b719ecc8e956a4f61c66efa76524122d36dc43d429372dd2a8530c6b879b56f7f05dd39828a4db3d08f9a7e3a329448c2ebcb07ceece7192981179024006eb10b546a87f51cc703a6c6bb32561e10d375b2fb063ecb30a88dc596581141e3def207d9c9266bd0b03df95952def951a9426d935290310cf4910c2c9cd4d024100a2a5f7c4ca07b459a1106998aa7c9177b106c22a2b1

In [52]:
# Make net address
idStr = 'z01x00'
pubHash = hashlib.sha3_224(pubKey).hexdigest()
address = idStr + pubHash
address

'z01x002d4a36d2a36c6a478308e2fd79e18bdca9a9e87f4b85bb08c84821e4'

In [43]:
# Sign msg
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256

signKey = RSA.import_key(prKey)
msg = 'To be signed'
msg = msg.encode('utf-8')
hashedMsg = SHA256.new(msg)
signature = pkcs1_15.new(signKey).sign(hashedMsg).hex()
signature

'4db3566bd03adcbc7d6556e4cddb323ae63c169c66b01c5088d189b9365d59783bcd9d9d67b1dbb4b12f99f614c08d5bad54f440346d902ff008edede459dba971e2bf1b71307c0880ec517a14c4dc1653fbc691fb32a17d15ab848531e59497fb49b3bdb8c813ff6c0eec37ef51727d7c011d5d569f29af6d67dc695a8cbfa8'

In [39]:
# Verify address hash
pubKeyFromAddress = address.split(idStr)[1]
if hashlib.sha3_256(pubKey).hexdigest() == pubKeyFromAddress:
    print('Yeeey!')
    
else:
    print('Not valid publick key')

Yeeey!


In [48]:
# Verify signiture
msgToVerif = 'To be signed'
msgToVerif = msgToVerif.encode('utf-8')

verifKey = RSA.import_key(pubKey)
h = SHA256.new(msgToVerif)

try:
    pkcs1_15.new(verifKey).verify(h, bytes.fromhex(signature))
    print('Valid')
except (ValueError, TypeError):
    print('Nice try...')

Valid


In [2]:
istr = Config().IDSTR
istr

'z01x00'

In [4]:
uuid.uuid4().hex

'626e0ec4cdb441beb9bc53af2dd78faa'

In [5]:
import uuid
import hashlib

from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_v1_5

from config import Config


def createWallet(password):
    # Create wallet ID
    uid = uuid.uuid4().hex
    
    # Generate keys pair
    key = RSA.generate(1024)
    # Export public and private keys and save them
    pubKey = key.publickey().export_key(format='DER')
    with open(os.path.join(os.path.join(Config().BASEDIR, 'keys'), f'{uid}_pubKey.der'), 'wb') as pubFile:
        pubFile.write(pubKey)
    
    prKey = key.export_key(format='DER', passphrase=password, pkcs=8,
                              protection="scryptAndAES128-CBC")
    with open(os.path.join(os.path.join(Config().BASEDIR, 'keys'), f'{uid}_prKey.der'), 'wb') as prFile:
        prFile.write(prKey)
        
    # Form address and return it to the user
    pubHash = hashlib.sha3_224(pubKey).hexdigest()
    address = Config().IDSTR+pubHash
    
    return address

In [6]:
password = 'lalalend'
address = createWallet(password)
address

'z01x00081671f35daba740645359878a47a78ffd8f34c9a1daed3bc46b5feb'

In [1]:
import os
import json

from Crypto.PublicKey import RSA
from config import Config

def authoriseUser(password):
    
    # Get keys names
    keysNames = os.listdir(os.path.join(Config().BASEDIR, 'keys'))
    # Get private key name
    prKeyName = [name for name in keysNames if name.split('_')[1] == 'prKey.der'][0]
    
    # Get and decrypt private key
    try:
        with open(os.path.join(os.path.join(Config().BASEDIR, 'keys'), prKeyName), 'rb') as keyFile:
            key = RSA.import_key(keyFile.read(), passphrase=password)
    except:
        return json.dumps({"MSG": "Wrong password"})
    
    prKey = key.export_key()
    
    return prKey

In [2]:
prKey = authoriseUser('lalalend')
prKey

'{"MSG": "Wrong password"}'

In [28]:
# Sign msg
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256

def signTransaction(msg, prKey):
    
    msg = msg.encode('utf-8')
    
    signKey = RSA.import_key(prKey)
    hashedMsg = SHA256.new(msg)
    signature = pkcs1_15.new(signKey).sign(hashedMsg).hex()
    
    return signature

In [29]:
msg = 'message'
sig = signTransaction(msg, prKey)
sig

'4763445dd69c805754544247ac99f835aef488593938a9d01ecd82d63c0febc27376ff8d72323102e96bac5216769f9c5dedd9c9fdf7d664f33a84f1280728843716f89784adec0904aa3a391ba1732126af61cc53af410c50ddc4f3ef289ec081cfbe6048e9ae6fdd3b529db3c03edb6ffe15b5e0f8c7d48161776c3f2fe61f'

In [37]:
# Verify address hash and signature
import hashlib 
import json

from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15

from config import Config
    
def verifyTxSignature(address, pubKey, msg, signature):
    
    pubKeyFromAddress = address.split(Config().IDSTR)[1]
    if hashlib.sha3_224(pubKey).hexdigest() == pubKeyFromAddress:
        
        msg = msg.encode('utf-8')
        verifKey = RSA.import_key(pubKey)
        msgHash = SHA256.new(msg)

        try:
            pkcs1_15.new(verifKey).verify(msgHash, bytes.fromhex(signature))
            return json.dumps({'MSG':True})
        
        except (ValueError, TypeError):
            return json.dumps({'MSG':False})

    else:
        return json.dumps({'MSG': 'Invalid public address'})
            

In [41]:
# Get keys names
keysNames = os.listdir(os.path.join(Config().BASEDIR, 'keys'))
# Get public key name
prKeyName = [name for name in keysNames if name.split('_')[1] == 'pubKey.der'][0]
# Get and decrypt public key
with open(os.path.join(os.path.join(Config().BASEDIR, 'keys'), prKeyName), 'rb') as keyFile:
    key = RSA.import_key(keyFile.read())

pubKey = key.public_key().export_key(format='DER')
address = address
msg = 'message'
signature = sig

sigVerif = verifyTxSignature(address, pubKey, msg, signature)
sigVerif

'{"MSG": true}'

In [6]:
# Get keys names
keysNames = os.listdir(os.path.join(Config().BASEDIR, 'keys'))
# Get public key name
prKeyName = [name for name in keysNames if name.split('_')[1] == 'pubKey.der'][0]
# Get and decrypt public key
with open(os.path.join(os.path.join(Config().BASEDIR, 'keys'), prKeyName), 'rb') as keyFile:
    key = RSA.import_key(keyFile.read())

In [11]:
pubKey = key.export_key(format='DER')
pubKey

b'0\x81\x9f0\r\x06\t*\x86H\x86\xf7\r\x01\x01\x01\x05\x00\x03\x81\x8d\x000\x81\x89\x02\x81\x81\x00\xefn\xb4\xcc\xb7\xe8\xff\xc7\xa4?h\x15\x0f\n|4\xcf\x96U\xdc\xd8\xfaOU3\x0b\x12\xdd\xd5\tqg\x10m$\\\xb7\x93\xb4\x15\x13\xad\x18\xf5\xb3\x0cyI&tM\x08\xbb\x1c\x89\xd9\x17\xc8J\x98\x137Q\xd8\xee\xe5\x89\x14"\x1c9\xbeNP\x8e\xce\x1b6\xeb\x08\x93+\x88\x1c\x9f4\x18\x1d\xea$\x84\xb3^\x1f\x84\xa5\x90\xe8\xb7\x7f\xfe`\x1b\xb5\x86\x00o<\xea\xb1\xbe\xbf\xfa\x00:\\\xa3\xd4\xeeJ\x00\xeb&\x9a\x945\x12\x91\x02\x03\x01\x00\x01'

In [12]:
# Form address and return it to the user
pubHash = hashlib.sha3_224(pubKey).hexdigest()
address = Config().IDSTR+pubHash
address

'z01x00a61643807d1f0bfe6dbb609311de768fb41430479b6564ca5f2c4f06'

In [13]:
os.listdir(os.path.join(Config().BASEDIR, 'keys')).split('_')[1]

AttributeError: 'list' object has no attribute 'split'

### Set Internet Broadcasting System

In [3]:
# Get public IP
publicIp = requests.get("https://api.ipify.org").content.decode()
publicIp

'82.151.196.144'

In [4]:
# Get local IP
socket.gethostbyname(socket.gethostname())

'192.168.0.16'

In [6]:
#Setup Alice's p2p node.
alice = Net(passive_bind="192.168.0.16", passive_port=44444, interface="eth0:2", node_type="passive", debug=1)
alice.start()
alice.bootstrap()
alice.advertise()

#Event loop.
while 1:
    for con in alice:
        for reply in con:
            print(reply)

    time.sleep(1)

Exception: Unable to connect to rendezvous server.

In [1]:
import upnpy

In [5]:
upnp = upnpy.UPnP()
device = upnp.get_igd()

IGDError: No IGD found.

In [3]:
upnp.AddPortMapping(
    NewRemoteHost='',
    NewExternalPort=80,
    NewProtocol='TCP',
    NewInternalPort=8000,
    NewInternalClient='192.168.0.16',
    NewEnabled=1,
    NewPortMappingDescription='Test port mapping entry from UPnPy.',
    NewLeaseDuration=0
) 

AttributeError: 'UPnP' object has no attribute 'AddPortMapping'

In [6]:
# upnp = upnpy.UPnP()

devices = upnpy.discover()
devices

ExpatError: not well-formed (invalid token): line 1, column 6

In [ ]:
# Open port
import miniupnpc

def open_port(port_no):
    '''this function opens a port using upnp'''
    upnp = miniupnpc.UPnP()

    upnp.discoverdelay = 10
    upnp.discover()

    upnp.selectigd()

    # addportmapping(external-port, protocol, internal-host, internal-port, description, remote-host)
    result=upnp.addportmapping(port_no, 'TCP', upnp.lanaddr, port_no, 'testing', '')
    return result

### Test Blockchain File System

In [9]:
host = 'http://0.0.0.0:5000'
# host = 'http://82.151.196.144:5000'

In [8]:
txPool = requests.get(host+'/getTxPool')
txPool

KeyboardInterrupt: 

In [ ]:
txPool.content

In [30]:
chain = requests.get(host+'/chain').content
chain

b'{"chain":[{"index":1,"previousHash":"9c3b78de7eb0066418a682aec7d0e6841cd5cbe70d71ae65309d4917cd1c5681","proof":42,"timestamp":1629032178.368946,"transactions":["ascl,sdc,d[v,[d,vsdcaaxasx]]","lv,sldc,sdc ;,d;d"]}],"length":1}\n'

In [20]:
nodes = requests.get(host+'/nodes/getNodes').content
nodes

b'{"nodes": ["0.0.0.0:5001"]}'

In [15]:
newNode = requests.post(host+'/nodes/register', json={'nodes':['http://0.0.0.0:5001']}).content
newNode

b'{"message":"New nodes have been added","total_nodes":["0.0.0.0:5001"]}\n'

In [61]:
respond = requests.get(host+'/nodes/getChainFilesAmount').content
respond

b'{"MSG":2}\n'

In [66]:
respond = requests.post(host+'/nodes/sendChainData', json={'iter': 0}).headers
respond

{'Content-Disposition': 'attachment; filename=1629031331.json', 'Content-Length': '361', 'Content-Type': 'application/json', 'Last-Modified': 'Sun, 15 Aug 2021 12:42:11 GMT', 'Cache-Control': 'public, max-age=43200', 'Expires': 'Tue, 17 Aug 2021 00:47:54 GMT', 'ETag': '"1629031331.106267-361-3308983641"', 'Server': 'Werkzeug/0.16.0 Python/3.7.2', 'Date': 'Mon, 16 Aug 2021 12:47:54 GMT'}

In [75]:
respond = requests.post(host+'/nodes/sendChainData', json={'iter': 1})
respond

<Response [200]>

In [81]:
fileName = re.split(r'; filename=', respond.headers['Content-Disposition'])[1]
fileName

'1629032179.json'

In [46]:
if request.status_code == 200:
    print('good')

good


In [85]:
lst = []
s = {i for i in lst if len(lst)>0}
s

set()

In [86]:
for node in s:
    print(node)

### Test New Transactions System

In [134]:
tx = {
    'type': 'common',
    'symbol': 'btc',
    'contract': 'btc',
    'sender': 'Selderey',
    'recipient': 'Makovka',
    'sendAmount':6.22,
    'comissionAmount': 3 
}

request = requests.post('http://0.0.0.0:5000/transactions/new', json=tx).content
request

b'{"msg":"Transaction will be added to Block 2"}\n'

In [158]:
txPool = requests.get('http://0.0.0.0:5000/getTxPool').content
txPool

b'{"txPool":[]}\n'

In [175]:
mineRequest = requests.get('http://0.0.0.0:5000/mine').content
mineRequest

b'{"index":2,"message":"New Block Forged","previous_hash":"468095a1151ef337794e03ec6ad360fcc305b24a32bd1ea01a1b0fb9267ff712","proof":35293,"transactions":[{"comissionAmount":1.0,"contract":null,"price":0.0,"recieveAmount":null,"recipient":"6257bc266e88480cbcd6418ab0dfa39d","sendAmount":1,"sender":"0","symbol":"zsh","timestamp":1628685557.224318,"tradeTxId":null},{"comissionAmount":1.0,"contract":"btc","price":30000.0,"recieveAmount":30000.0,"recipient":"him","sendAmount":1.0,"sender":"me","symbol":"btc/usdt","timestamp":1628685557.224583,"tradeTxId":"59af6324d5a53eba8a56243530ea2c25f1ca224a65eed04c0644f74e7347c38e"},{"comissionAmount":0.9090909090909091,"contract":"usdt","price":30000.0,"recieveAmount":1.0,"recipient":"me","sendAmount":30000.0,"sender":"him","symbol":"btc/usdt","timestamp":1628685557.22459,"tradeTxId":"f3f99e129e2e1691c7e7eeb8f0e4f1df4b5592e9f82d935f8d36f62cd3009c06"}]}\n'

In [173]:
tradeTx = {
    'type': 'trade',
    'sender': 'me',
    'symbol': 'btc/usdt',
    'price': 30000.0,
    'send': 'btc',
    'sendVol': 2.0,
    'get': 'usdt',
    'getVol': 60000.0,
    'comissionAmount':2
}

request = requests.post('http://0.0.0.0:5000/transactions/new', json=tradeTx).content
request

b'{"msg":"Transaction will be added to Block 2"}\n'

In [174]:
tradeTx = {
    'type': 'trade',
    'sender': 'him',
    'symbol': 'btc/usdt',
    'price': 33000.0,
    'send': 'usdt',
    'sendVol': 33000.0,
    'get': 'btc',
    'getVol': 1.0,
    'comissionAmount':1
}

request = requests.post('http://0.0.0.0:5000/transactions/new', json=tradeTx).content
request

b'{"msg":"Transaction will be added to Block 2"}\n'

In [176]:
ordersPool = requests.get('http://0.0.0.0:5000/getTradeOrders').content
ordersPool

b'{"tradeOrders":[{"comissionAmount":2.0,"get":"usdt","getVol":30000.0,"price":30000.0,"send":"btc","sendVol":1.0,"sender":"me","symbol":"btc/usdt","timestamp":1628685550.766065,"tradeTxId":"59af6324d5a53eba8a56243530ea2c25f1ca224a65eed04c0644f74e7347c38e"}]}\n'

### Orders Pool

In [53]:
testLst = [ {
    'sender': 'boy',
    'symbol': 'btc/dai',
    'price': 30000.0,
    'send': 'btc',
    'sendVol': 1.0,
    'get': 'dai',
    'getVol': 30000.0,
   
},

 {
    'sender':'man',
     'symbol': 'btc/dai',
     'price': 33333.33,
    'send': 'dai',
    'sendVol': 15000.0,
    'get': 'btc',
    'getVol': 0.45,
   
},

 {
    'sender':'girl',
     'symbol': 'eth/dai',
     'price': 2600.0,
    'send': 'eth',
    'sendVol': 1.0,
    'get': 'dai',
    'getVol': 2600.0,
   
},

 {
    'sender':'woman',
     'symbol': 'eth/dai',
     'price': 2750.0,
    'send': 'dai',
    'sendVol': 1650.0,
    'get': 'eth',
    'getVol': 0.6,
   
},
{
    'sender':'oldlady',
    'symbol': 'zcash/dai',
    'price': 500.0,
    'send': 'zcash',
    'sendVol': 1.0,
    'get': 'dai',
    'getVol': 500.0,
   
},
{
    'sender':'bug',
    'symbol': 'uni/dai',
    'price': 24.0,
    'send': 'uni',
    'sendVol': 1.0,
    'get': 'dai',
    'getVol': 24.0,
},
{
    'sender':'loft',
    'symbol': 'uni/usdt',
    'price': 24.0,
    'send': 'usdt',
    'sendVol': 24.0,
    'get': 'uni',
    'getVol': 1.0,
}
]

testLst

[{'sender': 'boy',
  'symbol': 'btc/dai',
  'price': 30000.0,
  'send': 'btc',
  'sendVol': 1.0,
  'get': 'dai',
  'getVol': 30000.0},
 {'sender': 'man',
  'symbol': 'btc/dai',
  'price': 33333.33,
  'send': 'dai',
  'sendVol': 15000.0,
  'get': 'btc',
  'getVol': 0.45},
 {'sender': 'girl',
  'symbol': 'eth/dai',
  'price': 2600.0,
  'send': 'eth',
  'sendVol': 1.0,
  'get': 'dai',
  'getVol': 2600.0},
 {'sender': 'woman',
  'symbol': 'eth/dai',
  'price': 2750.0,
  'send': 'dai',
  'sendVol': 1650.0,
  'get': 'eth',
  'getVol': 0.6},
 {'sender': 'oldlady',
  'symbol': 'zcash/dai',
  'price': 500.0,
  'send': 'zcash',
  'sendVol': 1.0,
  'get': 'dai',
  'getVol': 500.0},
 {'sender': 'bug',
  'symbol': 'uni/dai',
  'price': 24.0,
  'send': 'uni',
  'sendVol': 1.0,
  'get': 'dai',
  'getVol': 24.0},
 {'sender': 'loft',
  'symbol': 'uni/usdt',
  'price': 24.0,
  'send': 'usdt',
  'sendVol': 24.0,
  'get': 'uni',
  'getVol': 1.0}]

### Matching System

In [54]:
sybmolSortedLst = sorted([(testLst[i]['sender'],testLst[i]['symbol'],testLst[i]['send'],testLst[i]['get'],\
                           testLst[i]['price'],)\
                          for i in range(len(testLst))], key=itemgetter(1, 2))

sybmolSortedLst

[('boy', 'btc/dai', 'btc', 'dai', 30000.0),
 ('man', 'btc/dai', 'dai', 'btc', 33333.33),
 ('woman', 'eth/dai', 'dai', 'eth', 2750.0),
 ('girl', 'eth/dai', 'eth', 'dai', 2600.0),
 ('bug', 'uni/dai', 'uni', 'dai', 24.0),
 ('loft', 'uni/usdt', 'usdt', 'uni', 24.0),
 ('oldlady', 'zcash/dai', 'zcash', 'dai', 500.0)]

In [55]:
mathchedOrders = []
for i in range(len(sybmolSortedLst)-1):
    if (sybmolSortedLst[i][2] == sybmolSortedLst[i+1][3]) and (sybmolSortedLst[i][3] == sybmolSortedLst[i+1][2]) :
        
        # Split symbol and define direction and price matching
        splittedSymbol1 = re.split(r'/', sybmolSortedLst[i][1])[0]
        splittedSymbol2 = re.split(r'/', sybmolSortedLst[i+1][1])[0]
        
        if ((splittedSymbol1 == sybmolSortedLst[i][2] and splittedSymbol2 == sybmolSortedLst[i+1][3]) \
            and (sybmolSortedLst[i][-1] <= sybmolSortedLst[i+1][-1])) \
            or ((splittedSymbol1 == sybmolSortedLst[i][3] and splittedSymbol2 == sybmolSortedLst[i+1][2])\
               and (sybmolSortedLst[i][-1] >= sybmolSortedLst[i+1][-1])):
        
            mathchedOrders.append((sybmolSortedLst[i], sybmolSortedLst[i+1],))

mathchedOrders

[(('boy', 'btc/dai', 'btc', 'dai', 30000.0),
  ('man', 'btc/dai', 'dai', 'btc', 33333.33)),
 (('woman', 'eth/dai', 'dai', 'eth', 2750.0),
  ('girl', 'eth/dai', 'eth', 'dai', 2600.0))]

### Transaction System

In [56]:
commonTxs = []

# Get matched orders
txDir = {}
for i in range(len(mathchedOrders)):
    tempLst = [mathchedOrders[i][0][0], mathchedOrders[i][1][0]]
    txTempLst=[order for order in testLst if order['sender'] in tempLst]
    txDir[f'tx_{i}'] = txTempLst
    
# Calculate txs amount
txKeys = list(txDir.keys())
for j in range(len(txKeys)):
    tradeTxs = txDir[txKeys[j]]
    
    symbolSplitted = re.split(r'/', tradeTxs[0]['symbol'])[0]

    if tradeTxs[0]['send'] == symbolSplitted:

        tradePrice = tradeTxs[0]['price']
        seller = tradeTxs[0]
        buyer = tradeTxs[1]

        sellSendAddress = seller['sender']
        sellRecieveAddress = buyer['sender']
        sellSendToken = seller['send']
        sellSendAmount = min(seller['sendVol'], buyer['getVol'])

        buySendAddress = buyer['sender']
        buyRecieveAddress = seller['sender']
        buySendToken = seller['get']
        buySendAmount = sellSendAmount*tradePrice

        tradeSellTx = {
            'timestamp': time(),
            'symbol': seller['symbol'],
            'contract': symbolSplitted,
            'sender': sellSendAddress,
            'recipient': buySendAddress,
            'sendAmount': sellSendAmount,
            'recieveAmount': buySendAmount,
            'price': tradePrice
        }


        tradeBuyTx = {
            'timestamp': time(),
            'symbol': seller['symbol'],
            'contract': re.split(r'/', seller['symbol'])[1],
            'sender': buySendAddress,
            'recipient': sellSendAddress,
            'sendAmount': buySendAmount,
            'recieveAmount': sellSendAmount,
            'price': tradePrice
        }
        
        # Decrease order vols; Fill (aka delete from trade pool) order if vols == 0
        txDir[txKeys[j]][0]['sendVol'] -= sellSendAmount
        txDir[txKeys[j]][0]['getVol'] -= buySendAmount
        
        txDir[txKeys[j]][1]['sendVol'] -= buySendAmount
        txDir[txKeys[j]][1]['getVol'] -= sellSendAmount
        
        

    elif tradeTxs[0]['get'] == symbolSplitted:

        tradePrice = tradeTxs[0]['price']
        seller = tradeTxs[1]
        buyer = tradeTxs[0]

        sellSendAddress = seller['sender']
        sellRecieveAddress = buyer['sender']
        sellSendToken = seller['send']
        sellSendAmount = min(seller['sendVol'], buyer['getVol'])

        buySendAddress = buyer['sender']
        buyRecieveAddress = seller['sender']
        buySendToken = seller['get']
        buySendAmount = sellSendAmount*tradePrice

        tradeSellTx = {
            'timestamp': time(),
            'symbol': seller['symbol'],
            'contract': symbolSplitted,
            'sender': sellSendAddress,
            'recipient': buySendAddress,
            'sendAmount': sellSendAmount,
            'recieveAmount': buySendAmount,
            'price': tradePrice
        }


        tradeBuyTx = {
            'timestamp': time(),
            'symbol': seller['symbol'],
            'contract': re.split(r'/', seller['symbol'])[1],
            'sender': buySendAddress,
            'recipient': sellSendAddress,
            'sendAmount': buySendAmount,
            'recieveAmount': sellSendAmount,
            'price': tradePrice
        }
        
        # Decrease order vols; Fill (aka delete from trade pool) order if vols == 0
        txDir[txKeys[j]][1]['sendVol'] -= sellSendAmount
        txDir[txKeys[j]][1]['getVol'] -= buySendAmount
        
        txDir[txKeys[j]][0]['sendVol'] -= buySendAmount
        txDir[txKeys[j]][0]['getVol'] -= sellSendAmount
                 
       
    # Form tx and send it to common pool
    commonTxs.append(tradeSellTx)
    commonTxs.append(tradeBuyTx)
    
# Remove zero getVol transactions from tradeTxs pool  
txDirKeys = list(txDir.keys())
toRemove = []

for key in txDirKeys:
    toRemoveTemp = [order for i,order in enumerate(txDir[key]) if order['getVol']==0]
    if len(toRemoveTemp) > 0:
        toRemove.append(toRemoveTemp[0])

for removeOrder in toRemove:
    testLst.remove(removeOrder)

commonTxs


[{'timestamp': 1628673995.017817,
  'symbol': 'btc/dai',
  'contract': 'btc',
  'sender': 'boy',
  'recipient': 'man',
  'sendAmount': 0.45,
  'recieveAmount': 13500.0,
  'price': 30000.0},
 {'timestamp': 1628673995.017819,
  'symbol': 'btc/dai',
  'contract': 'dai',
  'sender': 'man',
  'recipient': 'boy',
  'sendAmount': 13500.0,
  'recieveAmount': 0.45,
  'price': 30000.0},
 {'timestamp': 1628673995.0178308,
  'symbol': 'eth/dai',
  'contract': 'eth',
  'sender': 'girl',
  'recipient': 'woman',
  'sendAmount': 0.6,
  'recieveAmount': 1560.0,
  'price': 2600.0},
 {'timestamp': 1628673995.017832,
  'symbol': 'eth/dai',
  'contract': 'dai',
  'sender': 'woman',
  'recipient': 'girl',
  'sendAmount': 1560.0,
  'recieveAmount': 0.6,
  'price': 2600.0}]

In [32]:
txDir

{'tx_0': [{'sender': 'boy',
   'symbol': 'btc/dai',
   'price': 30000.0,
   'send': 'btc',
   'sendVol': 0.55,
   'get': 'dai',
   'getVol': 16500.0},
  {'sender': 'man',
   'symbol': 'btc/dai',
   'price': 33333.33,
   'send': 'dai',
   'sendVol': 1500.0,
   'get': 'btc',
   'getVol': 0.0}],
 'tx_1': [{'sender': 'girl',
   'symbol': 'eth/dai',
   'price': 2600.0,
   'send': 'eth',
   'sendVol': 0.4,
   'get': 'dai',
   'getVol': 1040.0},
  {'sender': 'woman',
   'symbol': 'eth/dai',
   'price': 2750.0,
   'send': 'dai',
   'sendVol': 90.0,
   'get': 'eth',
   'getVol': 0.0}]}

In [57]:
testLst

[{'sender': 'boy',
  'symbol': 'btc/dai',
  'price': 30000.0,
  'send': 'btc',
  'sendVol': 0.55,
  'get': 'dai',
  'getVol': 16500.0},
 {'sender': 'girl',
  'symbol': 'eth/dai',
  'price': 2600.0,
  'send': 'eth',
  'sendVol': 0.4,
  'get': 'dai',
  'getVol': 1040.0},
 {'sender': 'oldlady',
  'symbol': 'zcash/dai',
  'price': 500.0,
  'send': 'zcash',
  'sendVol': 1.0,
  'get': 'dai',
  'getVol': 500.0},
 {'sender': 'bug',
  'symbol': 'uni/dai',
  'price': 24.0,
  'send': 'uni',
  'sendVol': 1.0,
  'get': 'dai',
  'getVol': 24.0},
 {'sender': 'loft',
  'symbol': 'uni/usdt',
  'price': 24.0,
  'send': 'usdt',
  'sendVol': 24.0,
  'get': 'uni',
  'getVol': 1.0}]

In [ ]:
for tx in txDir:

    toRemove = [order for i,order in enumerate(tx) if order['getVol']==0]

    for removeDict in toRemove:
        aList.remove(removeDict)

    aList

In [18]:
txDirKeys = list(txDir.keys())
toRemove = []

for key in txDirKeys:
    toRemoveTemp = [order for i,order in enumerate(txDir[key]) if order['getVol']==0]
    toRemove.append(toRemoveTemp[0])

for removeOrder in toRemove:
    testLst.remove(removeOrder)
    
testLst

[{'id': 'boy',
  'symbol': 'btc/dai',
  'price': 30000.0,
  'send': 'btc',
  'sendVol': 0.55,
  'get': 'dai',
  'getVol': 16500.0},
 {'id': 'girl',
  'symbol': 'eth/dai',
  'price': 2600.0,
  'send': 'eth',
  'sendVol': 0.4,
  'get': 'dai',
  'getVol': 1040.0},
 {'id': 'oldlady',
  'symbol': 'zcash/dai',
  'price': 500.0,
  'send': 'zcash',
  'sendVol': 1.0,
  'get': 'dai',
  'getVol': 500.0},
 {'id': 'bug',
  'symbol': 'uni/dai',
  'price': 24.0,
  'send': 'uni',
  'sendVol': 1.0,
  'get': 'dai',
  'getVol': 24.0},
 {'id': 'loft',
  'symbol': 'uni/usdt',
  'price': 24.0,
  'send': 'usdt',
  'sendVol': 24.0,
  'get': 'uni',
  'getVol': 1.0}]

In [7]:
tradeBuyTx

{'timestamp': 1628665190.98132,
 'symbol': 'eth/dai',
 'contract': 'dai',
 'sender': 'woman',
 'receiver': 'girl',
 'sentAmount': 1560.0,
 'recieveAmount': 0.6,
 'price': 2600.0}

In [8]:
tradeSellTx

{'timestamp': 1628418258.835157,
 'symbol': 'eth/dai',
 'contract': 'eth',
 'sender': 'girl',
 'receiver': 'woman',
 'sentAmount': 0.0,
 'recieveAmount': 0.0,
 'price': 2600.0}

In [111]:
txDir = {}
for i in range(len(mathchedOrders)):
    tempLst = [mathchedOrders[i][0][0], mathchedOrders[i][1][0]]
    txTempLst=[order for order in testLst if order['id'] in tempLst]
    txDir[f'tx_{i}'] = txTempLst

txDir

{'tx_0': [{'id': 'boy',
   'symbol': 'btc/dai',
   'price': 30000.0,
   'send': 'btc',
   'sendVol': 1.0,
   'get': 'dai',
   'getVol': 30000.0},
  {'id': 'man',
   'symbol': 'btc/dai',
   'price': 33333.33,
   'send': 'dai',
   'sendVol': 15000.0,
   'get': 'btc',
   'getVol': 0.45}],
 'tx_1': [{'id': 'girl',
   'symbol': 'eth/dai',
   'price': 2600.0,
   'send': 'eth',
   'sendVol': 1.0,
   'get': 'dai',
   'getVol': 2600.0},
  {'id': 'woman',
   'symbol': 'eth/dai',
   'price': 2750.0,
   'send': 'dai',
   'sendVol': 1650.0,
   'get': 'eth',
   'getVol': 0.6}]}

In [112]:
txKeys = list(txDir.keys())
firstTx = txDir[txKeys[1]]
firstTx

[{'id': 'girl',
  'symbol': 'eth/dai',
  'price': 2600.0,
  'send': 'eth',
  'sendVol': 1.0,
  'get': 'dai',
  'getVol': 2600.0},
 {'id': 'woman',
  'symbol': 'eth/dai',
  'price': 2750.0,
  'send': 'dai',
  'sendVol': 1650.0,
  'get': 'eth',
  'getVol': 0.6}]

In [113]:
symbolSplitted = re.split(r'/', firstTx[0]['symbol'])[0]

if firstTx[0]['send'] == symbolSplitted:
    
    tradePrice = firstTx[0]['price']
    seller = firstTx[0]
    buyer = firstTx[1]
    
    sellSendAddress = seller['id']
    sellRecieveAddress = buyer['id']
    sellSendToken = seller['send']
    sellSendAmount = min(seller['sendVol'], buyer['getVol'])
    
    buySendAddress = buyer['id']
    buyRecieveAddress = seller['id']
    buySendToken = seller['get']
    buySendAmount = sellSendAmount*tradePrice
    
    tradeSellTx = {
        'timestamp': time(),
        'symbol': seller['symbol'],
        'contract': symbolSplitted,
        'sender': sellSendAddress,
        'receiver': buySendAddress,
        'sentAmount': sellSendAmount,
        'recieveAmount': buySendAmount,
        'price': tradePrice
    }
    
    
    tradeBuyTx = {
        'timestamp': time(),
        'symbol': seller['symbol'],
        'contract': re.split(r'/', seller['symbol'])[1],
        'sender': buySendAddress,
        'receiver': sellSendAddress,
        'sentAmount': buySendAmount,
        'recieveAmount': sellSendAmount,
        'price': tradePrice
    }
    
    seller['sendVol'] -= sellSendAmount
    seller['getVol'] -= buySendAmount
    
    buyer['sendVol'] -= buySendAmount
    buyer['getVol'] -= sellSendAmount
    

    
elif firstTx[0]['get'] == symbolSplitted:
    
    tradePrice = firstTx[0]['price']
    seller = firstTx[1]
    buyer = firstTx[0]
    
    sellSendAddress = seller['id']
    sellRecieveAddress = buyer['id']
    sellSendToken = seller['send']
    sellSendAmount = min(seller['sendVol'], buyer['getVol'])
    
    buySendAddress = buyer['id']
    buyRecieveAddress = seller['id']
    buySendToken = seller['get']
    buySendAmount = sellSendAmount*tradePrice
    
    tradeSellTx = {
        'timestamp': time(),
        'symbol': seller['symbol'],
        'contract': symbolSplitted,
        'sender': sellSendAddress,
        'receiver': buySendAddress,
        'sentAmount': sellSendAmount,
        'recieveAmount': buySendAmount,
        'price': tradePrice
    }
    
    
    tradeBuyTx = {
        'timestamp': time(),
        'symbol': seller['symbol'],
        'contract': re.split(r'/', seller['symbol'])[1],
        'sender': buySendAddress,
        'receiver': sellSendAddress,
        'sentAmount': buySendAmount,
        'recieveAmount': sellSendAmount,
        'price': tradePrice
    }
    
    seller['sendVol'] -= sellSendAmount
    seller['getVol'] -= buySendAmount
    
    buyer['sendVol'] -= buySendAmount
    buyer['getVol'] -= sellSendAmount
    
tradeSellTx

{'timestamp': 1628253892.8598552,
 'symbol': 'eth/dai',
 'contract': 'eth',
 'sender': 'girl',
 'receiver': 'woman',
 'sentAmount': 0.6,
 'recieveAmount': 1560.0,
 'price': 2600.0}

In [114]:
tradeBuyTx

{'timestamp': 1628253892.8598568,
 'symbol': 'eth/dai',
 'contract': 'dai',
 'sender': 'woman',
 'receiver': 'girl',
 'sentAmount': 1560.0,
 'recieveAmount': 0.6,
 'price': 2600.0}

In [115]:
firstTx

[{'id': 'girl',
  'symbol': 'eth/dai',
  'price': 2600.0,
  'send': 'eth',
  'sendVol': 0.4,
  'get': 'dai',
  'getVol': 1040.0},
 {'id': 'woman',
  'symbol': 'eth/dai',
  'price': 2750.0,
  'send': 'dai',
  'sendVol': 90.0,
  'get': 'eth',
  'getVol': 0.0}]

In [116]:
# Check orders fullfillment
for order in firstTx:
    if order['sendVol'] == 0 or order['getVol'] == 0:
        order['status'] = 'filled'
        
firstTx

[{'id': 'girl',
  'symbol': 'eth/dai',
  'price': 2600.0,
  'send': 'eth',
  'sendVol': 0.4,
  'get': 'dai',
  'getVol': 1040.0},
 {'id': 'woman',
  'symbol': 'eth/dai',
  'price': 2750.0,
  'send': 'dai',
  'sendVol': 90.0,
  'get': 'eth',
  'getVol': 0.0,
  'status': 'filled'}]

In [14]:
tstAsk = sorted([(testLst[i]['id'],testLst[i]['ask'],testLst[i]['bid'],) for i in range(len(testLst))], key=itemgetter(1,2))
tstAsk

[('man', 'all', 'yel'),
 ('boy', 'bal', 'salt'),
 ('oldlady', 'moon', 'salt'),
 ('woman', 'salt', 'moon'),
 ('girl', 'yel', 'all'),
 ('bug', 'yel', 'all')]

In [9]:
tstBid = sorted([(testLst[i]['id'],testLst[i]['bid'],testLst[i]['ask'],) for i in range(len(testLst))], key=itemgetter(1,2))
tstBid

[(3, 'all', 'yel'),
 (6, 'all', 'yel'),
 (4, 'moon', 'salt'),
 (1, 'salt', 'bal'),
 (5, 'salt', 'moon'),
 (2, 'yel', 'all')]

In [10]:
mathchedOrders = []
c = 0
for getOrders, sendOrders in zip(tstToGet,tstToSend):
    if getOrders[1] == sendOrders[1] and getOrders[2] == sendOrders[2]:
        match = [getOrders, sendOrders]
        mathchedOrders.append(match)
        
mathchedOrders

[[(3, 'all', 'yel'), (2, 'all', 'yel')],
 [(4, 'moon', 'salt'), (5, 'moon', 'salt')],
 [(2, 'yel', 'all'), (6, 'yel', 'all')]]

In [12]:
toProcess = sorted([orders for orders in mathchedOrders], key=itemgetter(0,1))
toProcess

[[(2, 'yel', 'all'), (6, 'yel', 'all')],
 [(3, 'all', 'yel'), (2, 'all', 'yel')],
 [(4, 'moon', 'salt'), (5, 'moon', 'salt')]]

In [69]:
toPool = []

for i in range(len(mathchedOrders)-1):
    if mathchedOrders[i+1][1][0]==mathchedOrders[i][0][0]:
        toPool.append(mathchedOrders[i])
        
toPool

[[(3, 'all', 'yel'), (2, 'all', 'yel')]]

In [67]:
mathchedOrders[0][0][0]==mathchedOrders[1][1][0]

True

In [23]:
tstSort = sorted(testLst, key=lambda k: k['a'] == k['b'])
tstSort

[{'a': 'bal', 'b': 'salt'},
 {'a': 'all', 'b': 'yel'},
 {'a': 'yel', 'b': 'all'},
 {'a': 'salt', 'b': 'bal'}]

In [33]:
testLst

[{'id': 1, 'a': 'bal', 'b': 'salt'},
 {'id': 2, 'a': 'all', 'b': 'yel'},
 {'id': 3, 'a': 'yel', 'b': 'all'},
 {'id': 4, 'a': 'salt', 'b': 'bal'}]

In [58]:
nodes = requests.get('http://localhost:5000/nodes/getNodes').content
nodes

b'{"nodes": []}'

In [56]:
chain = requests.get('http://localhost:5000/nodes/resolve').content
# chainTrue = json.loads(chain)
# chainTrue
chain

b'{"chain":[{"index":1,"previousHash":1,"proof":100,"timestamp":1628086698.873197,"transactions":[]}],"message":"Our chain is authoritative"}\n'

In [53]:
regResult = requests.post('http://localhost:5000/nodes/register', json={'nodes': ('http://localhost:5000/', 'http://localhost:5001/',)}).content
regResult

b'{"message":"New nodes have been added","total_nodes":["localhost:5000","localhost:5001"]}\n'

In [35]:
chain = requests.get('http://localhost:5001/nodes/resolve').content
chain

b'{"chain":[{"index":1,"previousHash":1,"proof":100,"timestamp":1628084880.8870988,"transactions":[]},{"index":2,"previousHash":"653ad21073662fe7a06fdbb9e21a826076706f90556d2d66c56fe81bfa3d2145","proof":35293,"timestamp":1628085644.59333,"transactions":[{"amount":7,"recipient":"someone-other-address","sender":"lalalala"},{"amount":1,"recipient":"20a1f73aa3ba468dba631ddf901846e1","sender":"0"}]},{"index":3,"previousHash":"8d3b6a2ed7e2e91528035019185348775b148e3e4aad5a94de7c4068ae513611","proof":35089,"timestamp":1628085819.22687,"transactions":[{"amount":7,"recipient":"someone-other-address","sender":"lalalala"},{"amount":7,"recipient":"someone-other-address","sender":"lalalala"},{"amount":7,"recipient":"someone-other-address","sender":"lalalala"},{"amount":7,"recipient":"someone-other-address","sender":"lalalala"},{"amount":1,"recipient":"20a1f73aa3ba468dba631ddf901846e1","sender":"0"}]},{"index":4,"previousHash":"bbe0ad89914d2d99f9b640db2406e0696a44d5bcf63f05a8f25163e3df83d32d","proof

In [14]:
tx = {
    "sender": "d4ee26eee15148ee92c6cd394edd974e",
     "recipient": "someone-other-address",
     "amount": 19
}

txSent = requests.post("http://localhost:5000/transactions/new", json=tx).content
txSent

b'{"msg":"Transaction will be added to Block 2"}\n'

In [15]:
mining = requests.get('http://localhost:5000/mine').content
mining

b'{"index":2,"message":"New Block Forged","previous_hash":"5f0c3e03f04104c30131974c6fd4a0e970cf1506191b2bed6552ed2da5c8e091","proof":35293,"transactions":[{"amount":19,"recipient":"someone-other-address","sender":"d4ee26eee15148ee92c6cd394edd974e"},{"amount":19,"recipient":"someone-other-address","sender":"d4ee26eee15148ee92c6cd394edd974e"},{"amount":1,"recipient":"a05764f3f7014a54bc4c1a0ea4d83f37","sender":"0"}]}\n'

In [33]:
tx = {
    "sender": "lalalala",
     "recipient": "someone-other-address",
     "amount": 6
}

txSent = requests.post("http://localhost:5001/transactions/new", json=tx).content
txSent

b'{"msg":"Transaction will be added to Block 6"}\n'

In [34]:
mining = requests.get('http://localhost:5001/mine').content
mining

b'{"index":6,"message":"New Block Forged","previous_hash":"2ff99d3e69ae8bee2661a23f9a990bf33d867e1c7430fbae0a86bf6c0ee578d5","proof":43538,"transactions":[{"amount":6,"recipient":"someone-other-address","sender":"lalalala"},{"amount":1,"recipient":"20a1f73aa3ba468dba631ddf901846e1","sender":"0"}]}\n'